# Modellierung Windeinspeisung

## Pakete laden

In [1]:
# Laden von notwendigen Paketen
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib widget
import seaborn as sns
from sklearn import preprocessing
from sklearn.decomposition import PCA
from statsmodels import api as sm
import statsmodels.formula.api as smf


#standardeinstellungen
plt.rcParams['axes.xmargin'] = 0
pd.set_option('display.precision',2)
np.set_printoptions(precision=3)

## Daten einladen

Im Folgenden werden Realdaten einer Windkraftanlage mit einzelnen Sensoren geladen.
Ziel der Untersuchung ist die Nachbildung der Kennlinie und der daraus resultierenden Einspeisung in Abhängigkeit der Umgebungs- und Zustandsvariablen.

Die Daten werden in dem dataframe wind zusammengefasst:<br>
<img src="images/Winddaten.png" alt="multi" width="300" >

In [2]:
wind = pd.read_excel('Daten Wind Turbine.xlsb',sheet_name='Import',index_col=1,parse_dates=[1],engine='pyxlsb')
wind.info()
wind.drop(columns = "Wind_turbine_name",inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 210095 entries, 2013-01-01 00:00:00+01:00 to 2016-12-31 23:50:00+01:00
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Wind_turbine_name  210095 non-null  object 
 1   Pitch              209231 non-null  float64
 2   Rspeed             209231 non-null  float64
 3   Gspeed             209231 non-null  float64
 4   S_avg              209231 non-null  float64
 5   S_min              209231 non-null  float64
 6   S_max              209231 non-null  float64
 7   S_std              209231 non-null  float64
 8   P_avg              209231 non-null  float64
 9   P_min              209231 non-null  float64
 10  P_max              209231 non-null  float64
 11  P_std              209231 non-null  float64
 12  Q_avg              209231 non-null  float64
 13  Ws_avg             209231 non-null  float64
 14  Ws_min             209231 non-null  float64
 15  Ws_max       

Überprüfung des Datentyps des Index

In [3]:
print(wind.index.dtype)
# Das Datum wurde beim Einlesen nicht richtig erkannt (Datentyp O = Objekt)
# zweiter Versuch Datum parsen mit Umwandlung des Zeitformates inkl. Zeitumstellung in UCT-Zeit 
wind.index = pd.to_datetime(wind.index,utc = True)
print(wind.index.dtype)

object
datetime64[ns, UTC]


Fehldaten ausschließen

In [4]:
print(np.isnan(wind).sum())
index = np.nonzero(wind["S_avg"].isnull().values)
wind.iloc[index[0]].head()

Pitch      864
Rspeed     864
Gspeed     864
S_avg      864
S_min      864
S_max      864
S_std      864
P_avg      864
P_min      864
P_max      864
P_std      864
Q_avg      864
Ws_avg     864
Ws_min     864
Ws_max     864
Ws_std     864
Wdir      8228
Nadir     8228
Va_avg     864
Va_std     864
Temp       864
dtype: int64


,Pitch,Rspeed,Gspeed,S_avg,S_min,S_max,S_std,P_avg,P_min,P_max,P_std,Q_avg,Ws_avg,Ws_min,Ws_max,Ws_std,Wdir,Nadir,Va_avg,Va_std,Temp
Date_time,,,,,,,,,,,,,,,,,,,,,
2013-04-03 10:10:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-03 10:20:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-03 10:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-03 10:40:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-03 10:50:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
wind.dropna(inplace =True)

In [6]:
wind.head()

,Pitch,Rspeed,Gspeed,S_avg,S_min,S_max,S_std,P_avg,P_min,P_max,P_std,Q_avg,Ws_avg,Ws_min,Ws_max,Ws_std,Wdir,Nadir,Va_avg,Va_std,Temp
Date_time,,,,,,,,,,,,,,,,,,,,,
2012-12-31 23:00:00+00:00,-1.00,17.18,1800.99,1072.79,757.84,1399.74,116.39,1072.65,757.32,1399.74,116.47,14.49,8.96,6.48,11.26,0.81,180.25,176.44,3.78,7.31,5.09
2012-12-31 23:10:00+00:00,-1.00,17.17,1799.97,1061.75,661.90,1398.57,142.63,1061.43,660.95,1398.45,142.82,23.70,8.89,5.78,11.65,1.01,183.29,176.44,6.90,7.56,5.26
2012-12-31 23:20:00+00:00,-1.00,17.18,1801.19,1145.14,797.27,1529.85,162.52,1144.79,795.96,1529.84,162.69,25.48,9.19,6.10,11.33,1.01,185.08,176.44,8.63,7.49,5.56
2012-12-31 23:30:00+00:00,-1.00,17.18,1801.14,1184.32,764.87,1701.61,194.35,1183.98,763.56,1701.45,194.56,24.38,8.92,6.05,12.15,1.13,190.33,188.41,-2.60,12.38,5.70
2012-12-31 23:40:00+00:00,-0.98,17.18,1801.01,1317.69,819.66,1854.88,215.61,1317.55,818.96,1854.86,215.72,14.47,9.48,6.14,12.48,1.10,188.07,192.90,-4.78,9.61,5.82


## Deskritive Analyse

**Aufgabe:** deskriptive Analyse<br>
- Versuchen Sie entlang der erzeugten Abbildungen abzuleiten, ob die zugeschalteten "Einflussgrößen" wichtig bzw. nicht wichtig für die Beschreibung der Einspeisekennlinie sind
- Leiten sie die jeweilige Wirkungsweise auf die Einspeisekurve ab

### wesentliche Kenngrößen

wesentliche Kennzahlen

In [7]:
stats = wind.describe().T
stats

,count,mean,std,min,25%,50%,75%,max
Pitch,201867.0,9.80,23.38,-3.71e+01,-0.99,-0.97,0.02,262.61
Rspeed,201867.0,10.97,5.57,0.00e+00,9.24,12.01,15.65,17.22
Gspeed,201867.0,1152.08,583.38,-5.75e+02,972.07,1262.45,1642.20,1804.95
S_avg,201867.0,386.31,449.35,0.00e+00,51.58,220.63,561.55,2064.26
S_min,201867.0,228.73,278.18,0.00e+00,4.30,127.16,348.54,2029.73
S_max,201867.0,572.71,629.50,0.00e+00,93.31,324.87,873.65,2307.04
S_std,201867.0,80.63,100.01,0.00e+00,10.80,39.25,111.27,1013.35
P_avg,201867.0,383.99,450.57,-1.82e+01,42.51,219.41,561.17,2051.38
P_min,201867.0,223.95,280.59,-9.36e+01,-0.15,124.90,347.22,2029.61
P_max,201867.0,570.76,630.78,-1.45e+01,86.99,323.73,873.60,2306.51


### Boxplot der Daten

In [8]:
# Zuordnung zur gruppe anhand der Maximalausprägung 
klasse = [0,50, 400, 3000]
fig, axes =plt.subplots(nrows=1,ncols=3,**{'figsize': (14, 5)})
for i in range(0,3):
    lab=np.nonzero((stats["max"].values>klasse[i])&(stats["max"].values<klasse[i+1]))
    sns.boxplot(data=wind.iloc[:,lab[0]],ax=axes[i]);

### Windgeschwindigkeit vs. Erzeugung und Bereinigung der Daten-Fehler

In [9]:
# Darstellung des Datensatzes
fig, ax =plt.subplots(nrows=1,ncols=2,**{'figsize': (14, 5)})
lin=ax[0].scatter(wind["Ws_avg"],wind["S_avg"]);
ax[0].set(ylabel="Erzeugung [kW]",xlabel="Windgeschwindigkeit [m/s]",title ="Orginalzeitreihe");

# Bereinigung um Datenfehler (obere Abweichung)
raster = np.arange(5,12,.15)
for i,j in enumerate(raster[:-1]):
    zeig1,=np.nonzero((wind["Ws_avg"].values>raster[i]) & (wind["Ws_avg"].values<=raster[i+1]))
    wind.iloc[zeig1]["S_avg"]
    zeig2,=np.nonzero(wind.iloc[zeig1]["S_avg"].values>(200+(wind.iloc[zeig1]["S_avg"]).quantile(q=.98)))
    if zeig2.size >0:
        wind.drop(wind.index[zeig1[zeig2]],inplace=True)
lin=ax[1].scatter(wind["Ws_avg"],wind["S_avg"]);
ax[1].set(ylabel="Erzeugung [kW]",xlabel="Windgeschwindigkeit [m/s]",title="bereinigte Zeitreihe");

### Einfluss Anstellwinkel auf die Erzeugung

<img src="images/pitch.png" alt="multi" width="1200" >

Hintergrund:<br>
- Bei sehr schwachem Wind (unter 2,5 m/s) produziert die  Windenergieanlage keinen elektrischen Strom: Der Wind ist zu schwach, um die Rotorwelle anzutreiben. Die Blätter sind in so genannter  Fahnenstellung (Pitchwinkel ≈ 90°) gedreht. Die Windenergieanlage steht  still oder dreht sehr langsam, was Trudelbetrieb genannt wird.
- Bei normalem Wind (2,5 m/s bis 12 m/s) dreht die Windenergieanlage und  produziert Leistung, aber der Wind ist noch zu schwach, um die  Nennleistung der Anlage zu erreichen. Der Pitchwinkel ist 0°, die  Rotorblätter stehen im optimalen Arbeitspunkt. Von der Windleistung wird so viel wie möglich in mechanische Energie umgewandelt. Mit zunehmender Windgeschwindigkeit erhöht sich auch gleichermaßen die Drehzahl  („drehzahlvariabler Betrieb“), um die Schnelllaufzahl konstant und damit den Wirkungsgrad optimal zu halten.
- Bei Starkwind (12 m/s bis  25 m/s) ist die angebotene Windleistung zu groß und die Anlage muss in  ihrer Leistungsabgabe begrenzt werden. Die Anlage wird dann „gepitcht“.  Der Pitchwinkel nimmt mit der Windgeschwindigkeit zu (von 0° bis circa  30 °) und die Auftriebskraft wird so beeinflusst, dass die  Leistungsabgabe der Windenergieanlage konstant bei Nennleistung bleibt.
- Bei Sturm (ab 25 m/s) ist der Wind so stark, dass die Windenergieanlage  abgeschaltet werden muss, um eventuelle Schäden zu vermeiden. Der  Pitchwinkel ist nahezu 90°; die Blätter sind in Fahnenstellung.

Die Verdrehung der Blätter wird durch das Pitch-System realisiert. Da es  für jedes Rotorblatt als selbständiges und unabhängiges System  ausgeführt ist, können sie als drei Primärbremsen angesehen werden. Für  das sichere Herunterfahren der Anlage aus allen Zuständen reicht das  Verstellen von nur einem Rotorblatt, das in die Fahnenposition (Position in Richtung des Windes) gebracht wird.

In [10]:
def scatter_create(raster,zin,xin,yin,ax):
    lab=pd.Series(index=[*range(0,len(xin))],dtype="str")
    lab_order=list()
    for i,j in enumerate(raster[:-1]):
        zeig1,=np.nonzero((zin.values>raster[i]) & (zin.values<=raster[i+1]))
        lab[zeig1]=str(raster[i])+'-'+str(raster[i+1])
        lab_order.append(str(raster[i])+'-'+str(raster[i+1]))
    ax.set(xlabel=xin._name,ylabel=yin._name, title=yin._name+' über '+xin._name+' groupby: '+zin._name)
    g = sns.scatterplot(
    x=xin.values, y=yin.values,
    hue=lab,
    hue_order=lab_order,ax=ax);

### Einfluss Pitchwinkel auf die Erzeugung

In [11]:
fig, ax =plt.subplots(nrows=1,ncols=2,**{'figsize': (14, 5)})
# Histogramm der Pitchwinkel
lin=ax[0].hist(np.minimum(100,wind["Pitch"]),bins=20)
ax[0].set(xlabel='Pitch-Winkel°',ylabel='Anzahl',title = 'Histogramm Pitchwinkel');
# Einfluss Pitchwinkel/ Windgeschwindigkeit vs. Einspeisung**<br>                
raster =[-5, -1.5, 1, 5, 10, 15, 20, 30, 50, 300]
scatter_create(raster,wind["Pitch"],wind["Ws_avg"],wind["S_avg"],ax[1])

### Einfluss Turbulenz der Windgeschwindigkeit auf die Erzeugung

In [12]:
fig, ax =plt.subplots(nrows=1,ncols=2,**{'figsize': (14, 5)})
ax[0].scatter(wind["Ws_avg"],wind["Ws_std"]/wind["Ws_avg"])
ax[0].set(xlabel='Windgeschwindigkeit [m/s]',ylabel='Ws\_std / WS\_avg',title="Turbulenz über mittlere Windgeschwindigkeit");

raster =[-np.inf, 1, 1.25, 1.5, 2, 3, 4, 8]
scatter_create(raster,wind["Ws_std"],wind["Ws_avg"],wind["S_avg"],ax[1])

### Umgang mit Winkelangaben

**Windgeschwindigkeit vs. Windrichtung**

**Einfluss der Windrichtung auf die Erzeugung**

Erläuterung zum Feature-engineering: 
Die windrichtung wird in Grad auf dem Intervall [0-360] notiert.<br> 
Zwischen 0 und 360 liegt eine Sprungstelle, obwohl inhaltlich 1° und 360° nahezu indentische Windrichtungen sind. Das erlernen dieses Zusammenhangs ist für ein neuronales Netz möglich aber ressourcenaufwendig. Es erleichert dem neuronalem Netz die Verarbeitung, wenn ein feature-engineering so vorgenommen wird, dass die Features ohne Sprungstelle verarbeitet werden können.
Dies wird durch eine vektorielle Schreibweise der Wingeschwindigkeit ermöglicht, wodurch der  Winkels mit der Geschwindigkeit (Betrag des Vektors) kombiniert wird. 

In [13]:
fig, ax =plt.subplots(nrows=1,ncols=2,**{'figsize': (14, 7)})
lin=ax[0].scatter(wind["Wdir"],wind["Ws_avg"]);
ax[0].set(ylabel="Windgeschwindigkeit [m/s]",xlabel="Windrichtung [Degr]",title="Zusammenhang zwischen Windrichtung und Windgeschwindigkeit");

wind["Ws_avgx"]=wind["Ws_avg"]* np.cos(np.radians(wind["Wdir"]))
wind["Ws_avgy"]=wind["Ws_avg"]* np.sin(np.radians(wind["Wdir"]))
g = sns.scatterplot(
    x = wind["Ws_avgx"].values,
    y = wind["Ws_avgy"].values,
    hue=wind["S_avg"],ax=ax[1]
);
ax[1].set(xlabel="Ws_avgx",ylabel="Ws_avgy",title=" Einfluss der Windrichtung auf die Erzeugung") ;

**Einfluss der Fehlstellung der Gondel zum Wind auf die Erzeugung**

Bei der Fehlstellung der Gondel könnte ein äquivalenter Ansatz betrachtet werden. Da hier aber zu vermuten ist, dass positive wie negative Abweichungen sich gleichartig auswirken, verfolgen wir hier den Ansatz die Fehlstellung im Rahmen des feature-engineering als Absolutwert zu betrachten.

Hierzu müssen aber erst einmal die Winkel überprüft werden:<br>
<img src="images/winkel.png" alt="multi" width="800" >

In [14]:
# Betrachtung des dataframes
wind[["Wdir","Nadir", "Va_avg"]]

,Wdir,Nadir,Va_avg
Date_time,,,
2012-12-31 23:00:00+00:00,180.25,176.44,3.78
2012-12-31 23:10:00+00:00,183.29,176.44,6.90
2012-12-31 23:20:00+00:00,185.08,176.44,8.63
2012-12-31 23:30:00+00:00,190.33,188.41,-2.60
2012-12-31 23:40:00+00:00,188.07,192.90,-4.78
...,...,...,...
2016-12-31 22:10:00+00:00,195.14,195.14,-14.51
2016-12-31 22:20:00+00:00,196.18,196.18,-13.46
2016-12-31 22:30:00+00:00,195.72,195.72,-13.91


 erster Versuch: Va_avg, gegenüber der Differenz zwischen Wdir und Nadir

In [15]:
fig,ax =plt.subplots(1,1,**{'figsize':[14,6]})
ax.scatter(wind["Va_avg"].values,wind["Wdir"].values - wind["Nadir"].values);
ax.set(xlabel='Va_avg[°]',ylabel='Wdir - Nadir [°]');

zweiter Versuch: Beschränkung auf die Produktionszeiten

In [16]:
fig,ax =plt.subplots(1,1,**{'figsize':[14,6]})
# Auswertung nur für Betrieb
ind = wind.index[np.nonzero(wind["S_avg"].values>50)]
ax.scatter(wind.loc[ind]["Va_avg"].values,wind.loc[ind]["Wdir"].values - wind.loc[ind]["Nadir"].values);
ax.set(xlabel='Na_avg[°]',ylabel='Wdir - Nadir [°]');

dritter Versuch: Winkeldifferenz korrekt berechnen: 

In [17]:
# (bisherige Betrachtung Nadir=1° und Wdir =364° -> diff = -363°; Nadir=364° und Wdir =1° -> 364°)
# Anzatz verwendung des tan, da dieser periodisch ist
winkel = wind["Wdir"].values - wind["Nadir"].values
wind["Wdir-Nadir"] = np.rad2deg(np.arctan2(np.sin(np.radians(winkel)), np.cos(np.radians(winkel))))
fig,ax =plt.subplots(1,1,**{'figsize':[14,6]})
# Auswertung nur für Betrieb mit korrekter Winkeldifferenz
ax.scatter(wind.loc[ind]["Va_avg"].values,wind.loc[ind]["Wdir-Nadir"]);
ax.set(xlabel='Na_avg[°]',ylabel='Wdir - Nadir [°]');

In [18]:
# Betrachtung im Zeitablauf
fig,ax =plt.subplots(2,1,**{'figsize':[14,6]})
ax[1].plot(ind,wind.loc[ind]["Va_avg"].values);
ax[1].set(xlabel='Zeit',ylabel="Va_avg [°]")
ax[0].plot(ind,wind.loc[ind]["Wdir-Nadir"].values)
ax[0].set(xlabel='Zeit',ylabel="Wdir - Nadir [°]");

fünfter Versuch: Erklärung der Ausreißer mit Pitchwinkel und Verteilung auf Monate 

In [19]:
fig, ax =plt.subplots(nrows=1,ncols=2,**{'figsize': (14, 5)})
raster =[-5, -1.5, 1, 5, 10, 15, 20, 30, 50, 300]
scatter_create(raster,wind.loc[ind]["Pitch"],wind.loc[ind]["Va_avg"],wind.loc[ind]["Wdir-Nadir"],ax[0]);
# Aufteilung nach Monaten 
raster =list(range(1,13))
scatter_create(raster,ind.month,wind.loc[ind]["Va_avg"],wind.loc[ind]["Wdir-Nadir"],ax[1]);

Untersuchung der Abweichung in Abhängigkeit der Leistungsabgabe  

In [20]:
fig,ax =plt.subplots(1,2,**{'figsize':[14,6]})
ind = wind.index[np.nonzero((wind["S_avg"].values>50)&(wind.index.year<2015))]
ax[0].scatter(wind.loc[ind]["Ws_avg"].values,np.abs(wind.loc[ind]["Wdir-Nadir"]-wind.loc[ind]["Va_avg"]));
ax[0].set(xlabel='Ws_avg',ylabel='Wdir - Nadir [°]',title = "Fehlstellung Gondel in Abhängigkeit der Windgeschwindigkeit");

raster =[0, 5, 10, 15, 20, 45, 90, 200]
scatter_create(raster,wind["Va_avg"],wind["Ws_avg"],wind["S_avg"],ax[1])

**Einfluss der Temperatur auf die Erzeugung**

In [21]:
fig,ax =plt.subplots(1,2,**{'figsize':[14,6]})
raster =[-20, -5, -3, 0, 3, 5, 10, 15, 40]
scatter_create(raster,wind["Temp"],wind["Ws_avg"],wind["S_avg"],ax[0])
raster =[-20, 4, 40]
scatter_create(raster,wind["Temp"],wind["Ws_avg"],wind["S_avg"],ax[1])

**Zusammenfassung der Korrelation der Einflussfaktoren**

In [22]:
zeig = [1, 4, 13, 16, 17, 19, 21];

# Compute the correlation matrix
corr =np.round(100* wind.iloc[:,zeig].corr(),1)

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(8, 5))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0.5,annot =True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

## Modellierung der Windeinspeisung

Aufbereitung der Regressoren und Regressand des Trainingsdatensatzes (Windgeschwindigkeit und Scheinleistung)

Bereinigung um  Datensätze mit Pitchwinkel > 44° 

In [23]:
windbench = wind[wind["Pitch"]<44]

**Darstellung der normierten Daten**

In [24]:
fig,ax =plt.subplots(1,1,**{'figsize':[14,6]})
raster =[-5, -1.5, 1, 5, 10, 15, 20, 30, 50]
scatter_create(raster,windbench["Pitch"],windbench["Ws_avg"],windbench["S_avg"],ax)

In [25]:
plt.close()

**Datenvorbereitung**<br>
Partitionierung der Daten (Training, Validierung, Testdaten)

In [26]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(windbench, windbench["S_avg"], test_size=0.4)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

Fehler der Testdaten

Plot der Testdaten

In [27]:
#M0_spline.MAE = mean(abs(M0_spline.Ypred_Val - out.y(cv(:,2)&(WT.in.Pitch<44))));
#M0_spline.MAE

In [28]:
#fig, ax =plt.subplots(nrows=1,ncols=1,**{'figsize': (12, 7)})
#ax.scatter(M0_spline.X_Train,M0_spline.Y_Train)
#ax.set(xlabel='Windgeschwindigkeit [m/s]'',ylabel='Einspeisung [MWh]',title="Realität normiert");

## Erstellung eines KNN zur Prognose der Einspeisung

**Aufgabe 1:** Auswahl der Features

Erstellen Sie unterschiedliche Modelle, um die Relevanz der einzelnen Einflussgrößen (KNN.zeig) und deren Wirkungsweise zu analysieren
- Modell 1: Verwendung Eingangsgröße Windgeschwindigkeit
- Modell 2: Verwendung Eingangsgröße Windgeschwindigkeit und Pitch-Winkel
- Modell 3: Verwendung aller sinnvollen und relevanten Eingangsgrößen
Hinweis: Konstruktion Auswahl der Features  KNN.X = WT.in{:,[ a b c d]} 

- Vergleichen Sie den MAE der Validierungsdaten aus dem Modellvergleich mit dem Benchmark
    - Schauen Sie sich pro Modelloutput Figure 11-13 an
    - Figure 16: Zeitverlauf
    - Figure 17: Reproduktion der Einspeisung/ Windgeschwindigkeit
    - Figure 18: Fehler
- Was kann gut was noch nicht gut reproduziert werden? 
- Gibt es Systematische Abweichungen?
- Auswahl erklärende Variablen (Auswahl über zeig steuern)

In [29]:
x_train.head()

,Pitch,Rspeed,Gspeed,S_avg,S_min,S_max,S_std,P_avg,P_min,P_max,P_std,Q_avg,Ws_avg,Ws_min,Ws_max,Ws_std,Wdir,Nadir,Va_avg,Va_std,Temp,Ws_avgx,Ws_avgy,Wdir-Nadir
Date_time,,,,,,,,,,,,,,,,,,,,,,,,
2016-02-01 21:00:00+00:00,-0.99,16.64,1744.31,739.06,401.28,1168.46,171.74,738.85,400.51,1168.46,171.94,12.55,7.66,4.49,11.01,1.15,227.50,227.50,-0.81,10.14,8.48,-5.18,-5.65,0.00
2013-04-10 08:50:00+00:00,-0.93,16.84,1766.17,1109.62,369.77,1950.39,382.58,1104.68,344.60,1948.73,386.18,85.78,9.09,5.37,12.66,1.37,236.98,225.56,2.72,13.20,9.38,-4.95,-7.62,11.42
2015-06-02 05:20:00+00:00,-0.98,16.77,1758.04,854.55,351.13,1762.84,299.32,853.85,346.02,1762.64,299.85,22.75,8.07,5.38,11.72,1.21,211.38,211.38,4.55,12.56,14.07,-6.89,-4.20,0.00
2013-01-18 14:00:00+00:00,-1.00,10.41,1095.26,132.20,68.97,189.53,27.27,130.55,65.12,188.81,27.90,19.52,4.90,3.48,6.93,0.64,96.33,111.68,-15.35,14.39,-1.25,-0.54,4.87,-15.35
2014-02-14 10:20:00+00:00,-0.97,16.79,1761.16,967.97,441.84,1808.01,359.97,967.72,441.55,1807.77,360.12,9.15,8.31,5.49,11.66,1.42,153.54,158.08,-2.68,13.52,5.25,-7.44,3.70,-4.54


**Eingabe_Ausgabe-Objekt**

In [30]:
class KNN():
    pass

**data preprocessing**

In [31]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [32]:
KNN.scaler = 1
if KNN.scaler ==1:
    scalerX = MinMaxScaler()
    x_train_norm = scalerX.fit_transform(x_train)
    x_val_norm = scalerX.transform(x_val)
    x_test_norm = scalerX.transform(x_test)
    scalerY = MinMaxScaler()
    y_train_norm = scalerY.fit_transform(y_train.values.reshape(-1, 1))
    y_val_norm = scalerY.transform(y_val.values.reshape(-1, 1))
    y_test_norm = scalerY.transform(y_test.values.reshape(-1, 1))
else:
    scalerX = StandardScaler()
    x_train_norm = scalerX.fit_transform(x_train)
    scalerY = StandardScaler()
    y_train_norm = scalerY.fit_transform(y_train.values.reshape(-1, 1))
    y_val_norm = scalerY.transform(y_val.values.reshape(-1, 1))
    y_test_norm = scalerY.transform(y_test.values.reshape(-1, 1))

verwendete Features

In [33]:
wind.columns

Index(['Pitch', 'Rspeed', 'Gspeed', 'S_avg', 'S_min', 'S_max', 'S_std',
       'P_avg', 'P_min', 'P_max', 'P_std', 'Q_avg', 'Ws_avg', 'Ws_min',
       'Ws_max', 'Ws_std', 'Wdir', 'Nadir', 'Va_avg', 'Va_std', 'Temp',
       'Ws_avgx', 'Ws_avgy', 'Wdir-Nadir'],
      dtype='object')

**Erstellen Sie nun die Features mit hilfe des Zeigers KNN.zeig als Liste der relevanten Indizes des Dataframes winds**

In [ ]:
#KNN.zeig = [...,...,...];

In [ ]:
KNN.x_train = x_train_norm[:,KNN.zeig];
KNN.x_val = x_val_norm[:,KNN.zeig];
KNN.x_test = x_test_norm[:,KNN.zeig];

**Definition Features und Output**

In [ ]:
_,KNN.numFeatures = KNN.x_train.shape
KNN.numResponses = 1;

### Modellaufbau mit Keras-Sequential Schreibweise

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Modellaufbau
ANN = Sequential()
ANN.add(Dense(20,input_shape =[KNN.numFeatures], activation='relu'))
ANN.add(Dense(30,activation='relu'))
ANN.add(Dense(1, activation='linear'))
ANN.compile(loss ='mean_squared_error',optimizer='adam')
ANN.summary()

**Trainieren des neuronalen Netzes**

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
early = EarlyStopping(monitor='val_loss', patience=20)
check = ModelCheckpoint(filepath='windmodel.h5', 
                        monitor='val_loss', save_best_only=True)

history = ANN.fit(KNN.x_train,y_train_norm,epochs=1000, batch_size=5000,validation_data=(KNN.x_val,y_val_norm),callbacks=[early, check])

**Plot Verlauf der Verlustfunktion**

In [ ]:
fig, ax =plt.subplots(nrows=1,ncols=1,**{'figsize': (14, 7)})
lin1 = ax.plot(history.history['val_loss'],label='val_loss')
lin2 = ax.plot(history.history['loss'],label='loss')
ax.set(xlabel='Epoche',ylabel='loss')
ax.legend();
ax.grid()

**Anwendung auf die Testdaten: Berechnung Verlust**

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('windmodel.h5')
KNN.loss_train=ANN.evaluate(KNN.x_train,y_train_norm)
KNN.loss_val=ANN.evaluate(KNN.x_val,y_val_norm)
KNN.loss_test=ANN.evaluate(KNN.x_val,y_test_norm)

**Prognose von Training, Validierungs- und Testdaten**

In [ ]:
ypred_train_norm=ANN.predict(KNN.x_train)
ypred_val_norm=ANN.predict(KNN.x_val)
ypred_test_norm=ANN.predict(KNN.x_test)

**Inverse Scalierung**

In [ ]:
ypred_train = scalerY.inverse_transform(ypred_train_norm)
ypred_val = scalerY.inverse_transform(ypred_val_norm)
ypred_test = scalerY.inverse_transform(ypred_test_norm)

**Scatter Plot Orginal vs. Prognose**

In [ ]:
fig, ax =plt.subplots(nrows=1,ncols=3,**{'figsize': (14, 5)})
lin1=ax[0].scatter(x_train["Ws_avg"],y_train,label='orginal');
lin2=ax[0].scatter(x_train["Ws_avg"],ypred_train,label='pred');
ax[0].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Training',ylim=(0,2200));
ax[0].legend()
lin1=ax[1].scatter(x_val["Ws_avg"],y_val,label='orginal');
lin2=ax[1].scatter(x_val["Ws_avg"],ypred_val,label='pred');
ax[1].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Validierung',ylim=(0,2200));
ax[1].legend()
lin1=ax[2].scatter(x_test["Ws_avg"],y_test,label='orginal');
lin2=ax[2].scatter(x_test["Ws_avg"],ypred_test,label='pred');
ax[2].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Test',ylim=(0,2200));
ax[2].legend();

**Scatter des Fehlers**

In [ ]:
fig, ax =plt.subplots(nrows=1,ncols=3,**{'figsize': (14, 4)})
lin1=ax[0].scatter(x_train["Ws_avg"],ypred_train-y_train.values.reshape(-1,1),label='orginal');
ax[0].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Training',ylim=(-500,500));
ax[0].legend()
lin1=ax[1].scatter(x_val["Ws_avg"],ypred_val-y_val.values.reshape(-1,1),label='orginal');
ax[1].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Validierung',ylim=(-500,500));
ax[1].legend()
lin1=ax[2].scatter(x_test["Ws_avg"],ypred_test-y_test.values.reshape(-1,1),label='orginal');
ax[2].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Test',ylim=(-500,500));
ax[2].legend();

**Verlauf**

In [ ]:
fig, ax =plt.subplots(nrows=1,ncols=3,**{'figsize': (14, 5)})
lin1=ax[0].plot(x_train.index,y_train,label='orginal',marker ='.',linestyle='None');
lin2=ax[0].plot(x_train.index,ypred_train,label='pred',linestyle='None',marker ='.');
ax[0].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Training',ylim=(0,2200));
ax[0].legend()
lin1=ax[1].plot(x_val.index,y_val,linestyle='None',marker ='o',label='orginal');
lin2=ax[1].plot(x_val.index,ypred_val,linestyle='None',marker ='o',label='pred');
ax[1].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Validierung',ylim=(0,2200));
ax[1].legend()
lin1=ax[2].plot(x_test.index,y_test,linestyle='None',marker ='o',label='orginal');
lin2=ax[2].plot(x_test.index,ypred_test,linestyle='None',marker ='o',label='pred');
ax[2].set(ylabel="Leistung [MW]",xlabel="Windgeschwindigkeit [m/s]",title='Test',ylim=(0,2200));
ax[2].legend();

**Aufgabe 2: Einfluss Skalierungs- und Aktivierungsmethode**

- Variieren Sie den Parameters KNN.Scaler [ 0 1] und den verwendeten Layer der Aktvierung 
[relu; tanh; sigmoid].
- Wie wirkt sich dies auf das Ergebnis aus?

**Aufgabe 3: Netztopologie**
- Variieren Sie die Anzahl der hidden Layer 
- Die Anzahl der hidden units 
- Wie wirkt sich dies auf das Ergebnis aus?
- Wie reduzieren Sie die negativen Einspeisungen? (Hinweis: betrachten Sie die Möglichkeit in den Layern unterschiedliche Aktivierungsfunktionen zu nutzen

**Aufgabe 4: Zusatz Hyperparameter**
- Betrachten Sie die Einstellmöglichkeiten des Optimierers adam KNN.options
- Sie können einzelne Parameter individuell anders setzen und die Auswirkungen analysieren
- Untersuchen Sie zuerst die Veränderung der Lernrate